In [ ]:
import os

os.environ[
    "OMP_NUM_THREADS"
] = "64"  # for jupyter.nersc.gov otherwise the notebook only uses 2 cores

In [ ]:
from pathlib import Path
import healpy as hp
import matplotlib.pyplot as plt
import numpy as np
import pymaster as nmt
from astropy.io import fits

%matplotlib inline

In [ ]:
hp.disable_warnings()

In [ ]:
plt.style.use("seaborn-talk")

In [ ]:
import pysm3 as pysm
import pysm3.units as u

In [ ]:
nside = 2048
lmax = 2048

In [ ]:
comp = "IQU"

In [ ]:
components = list(enumerate(comp))
components

In [ ]:
spectra_components = ["TT", "EE", "BB"]

change this to True   if you want to  run namaster on notebook 


In [ ]:
namaster_on_nb = True

In [ ]:
datadir = Path("data/")

# Setting the inputs 
## Dust maps 
- We use the  2015 GNILC intensity map from the 2nd planck release, as it encodes less contamination from CIB with 21.8' resolution https://www.dropbox.com/s/hicocet83z31ob3/COM_CompMap_Dust-GNILC-F353_2048_21p8acm.fits?dl=0

- for Q and U we adopt maps from the 3rd Planck release as they were optimized for polarization studies with 80' reso.  



In [ ]:
dust_varresI = datadir / "COM_CompMap_Dust-GNILC-F353_2048_21p8acm.fits"

In [ ]:
if not dust_varresI.exists():
    !wget -O $dust_varresI https://portal.nersc.gov/project/cmb/pysm-data/gnilc/inputs/COM_CompMap_Dust-GNILC-F353_2048_21p8acm.fits

Transform maps to double precision for computations

In [ ]:
I_planck_varres, h = hp.read_map(dust_varresI, dtype=np.float64, h=True)

Maps from the two releases are in different units `MJy/sr` the former, and `K_CMB` the latter, we therefore need to perform some conversion to `uK_RJ`. 

In [ ]:
I_planck_varres <<= u.MJy / u.sr
I_planck_varres = I_planck_varres.to(
    "uK_RJ", equivalencies=u.cmb_equivalencies(353 * u.GHz)
)

## Amplitude modulation

In [ ]:
modulate_amp = hp.alm2map(
    hp.read_alm(
        datadir / "gnilc_dust_temperature_modulation_alms_lmax3072.fits",
    ).astype(np.complex128),
    nside=nside,
)

## Injecting small scales to Spectral parameters 

In [ ]:
tdfile = datadir / "COM_CompMap_Dust-GNILC-Model-Temperature_2048_R2.00.fits"
if not tdfile.exists():
    !wget -O $tdfile  http://pla.esac.esa.int/pla/aio/product-action?MAP.MAP_ID=COM_CompMap_Dust-GNILC-Model-Temperature_2048_R2.00.fits
bdfile = datadir / "COM_CompMap_Dust-GNILC-Model-Spectral-Index_2048_R2.00.fits"

if not bdfile.exists():
    !wget -O $bdfile http://pla.esac.esa.int/pla/aio/product-action?MAP.MAP_ID=COM_CompMap_Dust-GNILC-Model-Spectral-Index_2048_R2.00.fits

In [ ]:
td = hp.read_map(tdfile, dtype=np.float64)
bd = hp.read_map(bdfile, dtype=np.float64)

In [ ]:
cltd = hp.anafast(td, use_pixel_weights=True, lmax=lmax)
clbd = hp.anafast(bd, use_pixel_weights=True, lmax=lmax)

cl = {"bd": clbd, "td": cltd}
dust_params = list(cl.keys())
ell = np.arange(lmax + 1)

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def model(ell, A, gamma):
    out = A * ell ** gamma
    out[:2] = 0
    return out

In [ ]:
ell_fit_low = {"bd": 200, "td": 100}
ell_fit_high = {"bd": 400, "td": 400}

A_fit, gamma_fit, A_fit_std, gamma_fit_std = {}, {}, {}, {}
plt.figure(figsize=(25, 5))

for ii, pol in enumerate(dust_params):
    plt.subplot(131 + ii)
    xdata = np.arange(ell_fit_low[pol], ell_fit_high[pol])
    ydata = cl[pol][xdata]
    (A_fit[pol], gamma_fit[pol]), cov = curve_fit(model, xdata, ydata)

    A_fit_std[pol], gamma_fit_std[pol] = np.sqrt(np.diag(cov))
    plt.loglog(ell, cl[pol], label=" Anafast $C_\ell$")

    plt.plot(
        ell[ell_fit_low[pol] // 2 : ell_fit_high[pol] * 2],
        model(
            ell[ell_fit_low[pol] // 2 : ell_fit_high[pol] * 2],
            A_fit[pol],
            gamma_fit[pol],
        ),
        label="model fit",
    )

    plt.axvline(
        ell_fit_low[pol],
        linestyle="--",
        color="black",
        label="$ \ell={} $".format(ell_fit_low[pol]),
    )
    plt.axvline(
        ell_fit_high[pol],
        linestyle="--",
        color="gray",
        label="$ \ell={} $".format(ell_fit_high[pol]),
    )
    plt.legend()
    plt.grid()
    plt.title(
        f"{pol}   spectrum for dust   " + r"$\gamma_{fit}=$" + f"{gamma_fit[pol]:.2f}"
    )

    plt.ylabel("$ C_\ell $")
    plt.xlabel(("$\ell$"))
    plt.xlim(2, lmax)

- We inject smaller angular scales to the maps  by extrapolating the power law fitted from the GNILC spectral parameter maps 

- Smaller angular scales are modulated similarly as the intensity map in pol tens formalism. 

- the multipoles where the fit is performed are different given the observed spectra . In any case we don't fit beyond $\ell=400$, which is consistent with the TT analysis above 
- given the fact that we inject smaller angular scales with a steeper spectral index  than TT
$$\gamma_{\beta} = -1.96, \gamma_{Td} = -2.47, \gamma_{TT}= -1.29$$

we don't expect to injecti _small scale noise_ when rescaling at frequencies orders of magnitude lower or larger than  the reference one ( 353 GHz). 


In [ ]:
def sigmoid(x, x0, width, power=4):
    """Sigmoid function given start point and width
    Parameters
    ----------
    x : array
        input x axis
    x0 : float
        value of x where the sigmoid starts (not the center)
    width : float
        width of the transition region in unit of x
    power : float
        tweak the steepness of the curve
    Returns
    -------
    sigmoid : array
        sigmoid, same length of x"""
    return 1.0 / (1 + np.exp(-power * (x - x0 - width / 2) / width))

In [ ]:
np.random.seed(777)
# filter small scales
small_scales_input_cl = [
    1
    * model(ell, A_fit[pol], gamma_fit[pol])
    * (sigmoid(ell, ell_fit_high[pol], ell_fit_high[pol] / 10))
    for pol in dust_params
]
bd_ss_alm = hp.synalm(small_scales_input_cl[0], lmax=lmax)
td_ss_alm = hp.synalm(small_scales_input_cl[1], lmax=lmax)
alm_bd = hp.map2alm(bd, lmax=lmax, use_pixel_weights=True)
alm_td = hp.map2alm(td, lmax=lmax, use_pixel_weights=True)

bd_LS_alm = np.empty_like(alm_bd)
td_LS_alm = np.empty_like(alm_bd)
sig_func = sigmoid(ell, x0=ell_fit_high["bd"], width=ell_fit_high["bd"] / 10)
bd_LS_alm = hp.almxfl(alm_bd, np.sqrt(1.0 - sig_func))
td_LS_alm = hp.almxfl(alm_td, np.sqrt(1.0 - sig_func))

bd_ss = hp.alm2map(bd_ss_alm, nside=nside)
td_ss = hp.alm2map(td_ss_alm, nside=nside)

bd_ss *= modulate_amp
td_ss *= modulate_amp

bd_ls = hp.alm2map(bd_LS_alm, nside=nside)
td_ls = hp.alm2map(td_LS_alm, nside=nside)
bd_out = bd_ss + bd_ls
td_out = td_ss + td_ls

In [ ]:
cl_out = {
    "bd": hp.anafast(bd_out, use_pixel_weights=True, lmax=lmax),
    "td": hp.anafast(td_out, use_pixel_weights=True, lmax=lmax),
}

In [ ]:
for ii, pol in enumerate(dust_params):
    plt.loglog(ell, cl[pol], alpha=0.5, color="C%d" % ii)
    plt.loglog(ell, cl_out[pol], label=f" {pol}   ", color="C%d" % ii)

    plt.legend()
    plt.grid(True)
    plt.plot(
        ell[100:][2:],
        model(ell[100:], A_fit[pol], gamma_fit[pol])[2:],
        "--",
        color="red",
    )
    plt.axvline(ell_fit_high[pol], linestyle="--", color="gray")
    plt.axvline(100, linestyle="--", color="gray")

    plt.ylabel("$ C_\ell $")
    plt.xlabel(("$\ell$"))
    # plt.xlim(350, 500 )
    plt.xlim(2, lmax)

In [ ]:
lat = 35
cm = plt.cm.RdBu
plt.figure(figsize=(15, 8))
hp.gnomview(
    bd_out,
    title="Bd  w/ small scales ",
    rot=[0, lat],
    reso=3.75,
    cmap=cm,
    xsize=320,
    sub=234,
    min=1.2,
    max=1.9,
)
hp.gnomview(
    bd_ss, title="small scales ", rot=[0, lat], reso=3.75, xsize=320, cmap=cm, sub=232
)
hp.gnomview(
    I_planck_varres,
    title=" GNILC I MAP  ",
    rot=[0, lat],
    reso=3.75,
    cmap=cm,
    xsize=320,
    sub=233,
)

hp.gnomview(
    (bd),
    title=" Bd  GNILC  ",
    rot=[0, lat],
    reso=3.75,
    xsize=320,
    cmap=cm,
    sub=235,
    min=1.2,
    max=1.9,
)
hp.gnomview(
    bd_ls,
    title="  Bd large scales ",
    rot=[0, lat],
    reso=3.75,
    cmap=cm,
    xsize=320,
    sub=231,
    min=1.2,
    max=1.9,
)

plt.figure(figsize=(15, 8))
hp.gnomview(
    td_out,
    title="Td  w/ small scales ",
    rot=[0, lat],
    reso=3.75,
    xsize=320,
    cmap=cm,
    sub=234,
    min=15,
    max=25,
)
hp.gnomview(
    td_ss, title="small scales ", rot=[0, lat], reso=3.75, xsize=320, sub=232, cmap=cm
)
hp.gnomview(
    (modulate_amp),
    title=" modulation  ",
    rot=[0, lat],
    reso=3.75,
    xsize=320,
    cmap=cm,
    sub=233,
)

hp.gnomview(
    (td),
    title=" Td  GNILC  ",
    rot=[0, lat],
    reso=3.75,
    xsize=320,
    sub=235,
    cmap=cm,
    min=15,
    max=25,
)
hp.gnomview(
    td_ls,
    title="  Td large scales ",
    rot=[0, lat],
    reso=3.75,
    xsize=320,
    cmap=cm,
    sub=231,
    min=15,
    max=25,
)